## Noise Resilience Benchmarking of Hybrid Quantum-Classical Face Verification Under NISQ Constraints​

### Step 1：Image Filtering - from preprocess.py

In [1]:
# --- Imports & config ---
import os, random
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18

SEED = 123
random.seed(SEED); torch.manual_seed(SEED)

ROOT = Path("./")
DATA_A = ROOT / "data" / "positive"
DATA_B = ROOT / "data" / "negative"


IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

train_root = ROOT / "_bin_dataset" / "train"
val_root   = ROOT / "_bin_dataset" / "val"

def build_split(srcA, srcB, train_ratio=0.8, limit_per_class=300):
    import shutil
    for p in [train_root, val_root]:
        if p.exists():
            shutil.rmtree(p)
    for p in [train_root/srcA.name, train_root/srcB.name, val_root/srcA.name, val_root/srcB.name]:
        p.mkdir(parents=True, exist_ok=True)

    def list_imgs(d: Path):
        exts = {".jpg",".jpeg",".png",".bmp",".webp"}
        return [p for p in d.rglob("*") if p.suffix.lower() in exts]

    A = list_imgs(srcA)
    B = list_imgs(srcB)
    random.shuffle(A); random.shuffle(B)
    A = A[:min(limit_per_class, len(A))]
    B = B[:min(limit_per_class, len(B))]

    kA = int(len(A)*train_ratio)
    kB = int(len(B)*train_ratio)

    for src in A[:kA]: shutil.copy(src, train_root/srcA.name/src.name)
    for src in A[kA:]: shutil.copy(src, val_root/srcA.name/src.name)
    for src in B[:kB]: shutil.copy(src, train_root/srcB.name/src.name)
    for src in B[kB:]: shutil.copy(src, val_root/srcB.name/src.name)

build_split(DATA_A, DATA_B)

train_ds = datasets.ImageFolder(train_root, transform=tfm)
val_ds   = datasets.ImageFolder(val_root,   transform=tfm)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=0)

print("Classes:", train_ds.classes)


Classes: ['negative', 'positive']


In [ ]:
ckpt_backbone = ROOT / "outputs" / "resnet18_backbone_only.pt"
if not ckpt_backbone.exists():
    alt = ROOT / "resnet18_finetuned.pt"
    assert alt.exists(), "outputs/resnet18_backbone_only.pt or resnet18_finetuned.pt"
    ckpt_backbone = alt

backbone = resnet18(weights=None)
state = torch.load(ckpt_backbone, map_location="cpu")
if isinstance(state, dict) and "state_dict" in state:
    state = state["state_dict"]
_ = backbone.load_state_dict(state, strict=False)

in_feats = backbone.fc.in_features  # 512
backbone.fc = nn.Identity()
backbone.to(device).eval()
for p in backbone.parameters():
    p.requires_grad_(False)


## L512-4

In [ ]:
class L512to4(nn.Module):
    def __init__(self, in_dim=512, hidden_dim=4):
        super().__init__()
        self.fc = nn.Linear(in_dim, hidden_dim)
        self.act = nn.Tanh()

    def forward(self, z):  # z: [B,512]
        return self.act(self.fc(z))  # [B,4]

proj = L512to4(in_dim=in_feats, hidden_dim=4).to(device)

## Quantum Circuit

In [ ]:
import pennylane as qml
from pennylane import numpy as pnp

n_qubits = 4
n_layers = 6

dev = qml.device("default.qubit", wires=n_qubits)

def entangle_ladder():

    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

@qml.qnode(dev, interface="torch")
def quantum_block(x, weights):
    """
    x: [4] classical input from L512→4
    weights: [6, 4] quantum parameters per layer per qubit
    """
    for q in range(n_qubits):
        qml.Hadamard(wires=q)
        qml.RY(pnp.pi * x[q] / 2.0, wires=q)

    for l in range(n_layers):
        for q in range(n_qubits):
            qml.RY(weights[l, q], wires=q)
        entangle_ladder()

    return [qml.expval(qml.PauliZ(q)) for q in range(n_qubits)]


In [ ]:
#############################################
# QuantumLayer: input [B,4] → output [B,4]
#############################################
class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()
        w0 = 0.01 * torch.randn(n_layers, n_qubits)
        self.weights = nn.Parameter(w0)

    def forward(self, x4_batch):
        outs = []
        for i in range(x4_batch.shape[0]):
            y = quantum_block(x4_batch[i], self.weights)  # ← outputs are float64
            y = torch.stack(y)                            # shape [4], still float64
            outs.append(y)
        zq = torch.stack(outs, dim=0)                     # [B,4]

        zq = zq.to(torch.float32)

        return zq

q_layer = QuantumLayer().to(device)


## L4-2

In [ ]:
class L4to2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(4, 2)
    def forward(self, z4):
        return self.fc(z4)

head = L4to2().to(device)

## L+Q+L

In [ ]:
#############################################
# Hybrid model = backbone → L512→4 → Q → L4→2
#############################################
class HybridModel(nn.Module):
    def __init__(self, backbone, proj, q_layer, head):
        super().__init__()
        self.backbone = backbone
        self.proj = proj
        self.q_layer = q_layer
        self.head = head

    def forward(self, imgs):
        with torch.no_grad():
            z512 = self.backbone(imgs)

        x4 = self.proj(z512)            # [B,4]
        zq = self.q_layer(x4)           # [B,4], quantum output
        logits = self.head(zq)          # [B,2]

        return logits

model = HybridModel(backbone, proj, q_layer, head).to(device)


In [ ]:
crit = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([
    {"params": proj.parameters(), "lr": 1e-3},
    {"params": q_layer.parameters(), "lr": 1e-2},
    {"params": head.parameters(), "lr": 1e-3},
])

def run_epoch(loader, train=True):
    model.train(train)

    loss_sum = 0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            logits = model(imgs)
            loss = crit(logits, labels)

        if train:
            loss.backward()
            optimizer.step()

        loss_sum += loss.item() * imgs.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

    return loss_sum/total, correct/total

# ----------------- Run training -----------------
for ep in range(1, 6):
    trL, trA = run_epoch(train_loader, True)
    vaL, vaA = run_epoch(val_loader, False)
    print(f"[{ep}] Train {trL:.4f}/{trA:.3f} | Val {vaL:.4f}/{vaA:.3f}")


[1] Train 0.3837/0.929 | Val 0.2993/0.983
[2] Train 0.2883/0.975 | Val 0.2797/0.975
[3] Train 0.2639/0.981 | Val 0.2609/0.975
[4] Train 0.2424/0.983 | Val 0.2417/0.975
[5] Train 0.2290/0.981 | Val 0.2303/0.975


In [ ]:
import os, torch, json
from datetime import datetime

SAVE_DIR = "artifacts"
os.makedirs(SAVE_DIR, exist_ok=True)

num_classes = getattr(model, "num_classes", None) or 2
class_names = None
for cand in ["train_ds", "dataset", "train_set"]:
    if cand in globals() and hasattr(globals()[cand], "classes"):
        class_names = list(getattr(globals()[cand], "classes"))
        break
if class_names is None:
    class_names = [f"class{i}" for i in range(num_classes)]

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

ckpt = {
    "state_dict": model.state_dict(),
    "meta": {
        "class_names": class_names,
        "img_size": 224,
        "mean": IMAGENET_MEAN,
        "std": IMAGENET_STD,
        "model_hparams": {
            "proj_dim": 4,
            "n_qubits": 4,
            "n_layers": 6, 
            "num_classes": num_classes
        },
        "notes": "ResNet18 backbone + 512->4 tanh + 4q VQC + 4->2 head"
    },
    "versions": {
        "torch": torch.__version__
    },
    "saved_at": datetime.now().isoformat(timespec="seconds"),
}

save_path = os.path.join(SAVE_DIR, "hybrid_qml_best.pt")
torch.save(ckpt, save_path)
print(f"Saved checkpoint to: {save_path}")

with open(os.path.join(SAVE_DIR, "hybrid_qml_best.meta.json"), "w") as f:
    json.dump(ckpt["meta"], f, indent=2)

Saved checkpoint to: artifacts\hybrid_qml_best.pt


In [11]:
# === Save + reload + single-image predict (matches this notebook's paths/arch) ===
import os, torch
from pathlib import Path
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet18

# Reuse ROOT if defined; otherwise default to project root
ROOT = Path(globals().get("ROOT", Path("./")))
SAVE_PATH = ROOT / "best_model.pt"

# 1) Save the current trained model (state_dict)
if "model" in globals():
    torch.save(model.state_dict(), SAVE_PATH)
    print(f"[save] Saved model state_dict to: {SAVE_PATH}")
else:
    print("[save] No 'model' in memory; will just load from disk below.")

# 2) Rebuild the SAME hybrid architecture and load weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Require the same classes that exist earlier in this notebook
required = ["L512to4", "QuantumLayer", "L4to2", "HybridModel"]
missing = [cls for cls in required if cls not in globals()]
if missing:
    raise RuntimeError(f"Please run the earlier cells to define {missing} before running this cell.")

# Backbone (fc removed to yield 512-d features)
backbone = resnet18(weights=None)
backbone.fc = torch.nn.Identity()

proj = L512to4(in_dim=512, hidden_dim=4)
q_layer = QuantumLayer()
head = L4to2()

model_inf = HybridModel(backbone, proj, q_layer, head).to(device)
state = torch.load(SAVE_PATH, map_location=device)
model_inf.load_state_dict(state, strict=False)
model_inf.eval()
print("[load] Loaded weights and rebuilt hybrid model for inference.")

# 3) Preprocess (same as training/ImageNet stats)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
infer_tf = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# 4) Predict ONE image
@torch.no_grad()
def predict_one_with_model(img_path: str, model_for_infer: torch.nn.Module, class_names=None):
    img = Image.open(img_path).convert("RGB")
    x = infer_tf(img).unsqueeze(0).to(device)

    logits = model_for_infer(x)

    # binary or multi-class friendly
    if logits.shape[1] == 1:
        p1 = torch.sigmoid(logits[:, 0])
        probs = torch.stack([1 - p1, p1], dim=1)
    else:
        probs = F.softmax(logits, dim=1)

    probs = probs.squeeze(0).cpu()
    pred_idx = int(probs.argmax().item())

    # reuse dataset class names if available
    if class_names is None:
        names = None
        for cand in ["train_ds", "val_ds", "dataset", "train_set"]:
            if cand in globals() and hasattr(globals()[cand], "classes"):
                names = list(getattr(globals()[cand], "classes"))
                break
        if names is None or len(names) != probs.numel():
            names = [f"class{i}" for i in range(probs.numel())]
    else:
        names = class_names if len(class_names) == probs.numel() else [f"class{i}" for i in range(probs.numel())]

    print(f"Image: {img_path}")
    print(f"Prediction: {names[pred_idx]} (idx={pred_idx})")
    for i, p in enumerate(probs.tolist()):
        print(f"  {names[i]:>12s}: {p:.4f}")

    return {"label": names[pred_idx], "index": pred_idx, "probs": probs.tolist()}

[save] Saved model state_dict to: best_model.pt
[load] Loaded weights and rebuilt hybrid model for inference.
